<a href="https://colab.research.google.com/github/DrVenkataRajeshKumar/Rajesh/blob/master/EVA4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(8)
        self.drop1 = nn.Dropout2d(0.05)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(8)
        self.drop2 = nn.Dropout2d(0.05)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1, bias=False)
        self.bn3   = nn.BatchNorm2d(16)
        self.drop3 = nn.Dropout2d(0.05)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1, bias=False)
        self.bn4   = nn.BatchNorm2d(32)
        self.drop4 = nn.Dropout2d(0.05)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(32, 32, 3, bias=False)
        self.bn5   = nn.BatchNorm2d(32)
        self.drop5 = nn.Dropout2d(0.05)
        self.conv6 = nn.Conv2d(32, 10, 3, bias=False)
        self.gap   = nn.AvgPool2d(3)

    def forward(self, x):
        x = F.relu(self.drop1(self.bn1(self.conv1(x))))
        x = self.pool1(F.relu(self.drop2(self.bn2(self.conv2(x)))))
        x = F.relu(self.drop3(self.bn3(self.conv3(x))))
        x = self.pool2(F.relu(self.drop4(self.bn4(self.conv4(x)))))
        x = F.relu(self.drop5(self.bn5(self.conv5(x))))
        x = self.conv6(x)
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)


In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
         Dropout2d-3            [-1, 8, 28, 28]               0
            Conv2d-4            [-1, 8, 28, 28]             576
       BatchNorm2d-5            [-1, 8, 28, 28]              16
         Dropout2d-6            [-1, 8, 28, 28]               0
         MaxPool2d-7            [-1, 8, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           1,152
       BatchNorm2d-9           [-1, 16, 14, 14]              32
        Dropout2d-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 32, 14, 14]           4,608
      BatchNorm2d-12           [-1, 32, 14, 14]              64
        Dropout2d-13           [-1, 32, 14, 14]               0
        MaxPool2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm_notebook
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm_notebook(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.




Test set: Average loss: 0.0581, Accuracy: 9844/10000 (98.44%)





Test set: Average loss: 0.0375, Accuracy: 9894/10000 (98.94%)





Test set: Average loss: 0.0317, Accuracy: 9897/10000 (98.97%)





Test set: Average loss: 0.0249, Accuracy: 9925/10000 (99.25%)



...



Test set: Average loss: 0.0176, Accuracy: 9945/10000 (99.45%)





Test set: Average loss: 0.0177, Accuracy: 9950/10000 (99.50%)





Test set: Average loss: 0.0192, Accuracy: 9938/10000 (99.38%)





Test set: Average loss: 0.0167, Accuracy: 9947/10000 (99.47%)





Test set: Average loss: 0.0152, Accuracy: 9945/10000 (99.45%)

